In [2]:
import timm
import torch
import torch.nn as nn
import numpy as np

In [3]:
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

model = timm.create_model('vit_base_patch16_224', pretrained=True)

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

In [19]:
dict(dict(dict(model.named_children())['blocks'].named_children())['0'].named_children())['attn']

Attention(
  (qkv): Linear(in_features=768, out_features=2304, bias=True)
  (q_norm): Identity()
  (k_norm): Identity()
  (attn_drop): Dropout(p=0.0, inplace=False)
  (proj): Linear(in_features=768, out_features=768, bias=True)
  (proj_drop): Dropout(p=0.0, inplace=False)
)

In [5]:
layer = 0
block = dict(dict(dict(model.named_children())['blocks'].named_children())[str(layer)].named_children())
# list(block['attn'].children())[0]
ls = [list(block['attn'].children())[0], list(block['attn'].children())[4],list(block['mlp'].children())[0], list(block['mlp'].children())[4]]
ls[0].weight.clone().detach().numpy().T

Linear(in_features=768, out_features=2304, bias=True)

In [31]:
layer = 0
block = dict(dict(dict(model.named_children())['blocks'].named_children())[str(layer)].named_children())
ls = [block['norm1'],list(block['attn'].children())[0], list(block['attn'].children())[4],block['norm2'],list(block['mlp'].children())[0], list(block['mlp'].children())[4]]

weight_ls = list()
for i in range(len(ls)):
    weight_ls.append(ls[i].weight.clone().detach().numpy().T)
    weight_ls.append(ls[i].bias.clone().detach().numpy())
for i in weight_ls:
    print(i.shape)

(768,)
(768,)
(768, 2304)
(2304,)
(768, 768)
(768,)
(768,)
(768,)
(768, 3072)
(3072,)
(3072, 768)
(768,)


In [30]:
for layer in range(12):
    block = dict(dict(dict(model.named_children())['blocks'].named_children())[str(layer)].named_children())
    ls = [block['norm1'],list(block['attn'].children())[0], list(block['attn'].children())[4],block['norm2'],list(block['mlp'].children())[0], list(block['mlp'].children())[4]]

    weight_ls = list()
    for i in range(len(ls)):
        weight_ls.append(ls[i].weight.clone().detach().numpy().T)
        weight_ls.append(ls[i].bias.clone().detach().numpy())

    print(weight_ls[0].flatten().shape)
    blk_weight = np.array(weight_ls[0].flatten())

    for i in range(1, len(weight_ls)):
        print(weight_ls[i].flatten().shape)
        blk_weight = np.concatenate((blk_weight, weight_ls[i].flatten()))
        print(blk_weight.shape, "\n")
    blk_weight.tofile('./pre_weights/'+str(layer)+'_newblock'+'.bin')

(768,)
(768,)
(1536,) 

(1769472,)
(1771008,) 

(2304,)
(1773312,) 

(589824,)
(2363136,) 

(768,)
(2363904,) 

(768,)
(2364672,) 

(768,)
(2365440,) 

(2359296,)
(4724736,) 

(3072,)
(4727808,) 

(2359296,)
(7087104,) 

(768,)
(7087872,) 

(768,)
(768,)
(1536,) 

(1769472,)
(1771008,) 

(2304,)
(1773312,) 

(589824,)
(2363136,) 

(768,)
(2363904,) 

(768,)
(2364672,) 

(768,)
(2365440,) 

(2359296,)
(4724736,) 

(3072,)
(4727808,) 

(2359296,)
(7087104,) 

(768,)
(7087872,) 

(768,)
(768,)
(1536,) 

(1769472,)
(1771008,) 

(2304,)
(1773312,) 

(589824,)
(2363136,) 

(768,)
(2363904,) 

(768,)
(2364672,) 

(768,)
(2365440,) 

(2359296,)
(4724736,) 

(3072,)
(4727808,) 

(2359296,)
(7087104,) 

(768,)
(7087872,) 

(768,)
(768,)
(1536,) 

(1769472,)
(1771008,) 

(2304,)
(1773312,) 

(589824,)
(2363136,) 

(768,)
(2363904,) 

(768,)
(2364672,) 

(768,)
(2365440,) 

(2359296,)
(4724736,) 

(3072,)
(4727808,) 

(2359296,)
(7087104,) 

(768,)
(7087872,) 

(768,)
(768,)
(1536,) 

(1769472,)
(

- norm1
- Wqkv
- proj
- norm2
- mlp1
- mlp2

<img src="./layer_norm.png" title="layernorm"/>

In [ ]:
weight_ls = list()
for i in range(len(ls)):
    weight_ls.append(ls[i].weight.clone().detach().numpy().T)
    weight_ls.append(ls[i].bias.clone().detach().numpy())

# qkv_w = qkv.weight.clone().detach().numpy().T
# qkv_w.tofile('./pre_weights/qkv_W_head_'+str(layer)+'.bin')
print(weight_ls[0].flatten().shape)
blk_weight = np.array(weight_ls[0].flatten())

for i in range(1, len(weight_ls)):
    print(weight_ls[i].flatten().shape)
    blk_weight = np.concatenate((blk_weight, weight_ls[i].flatten()))
    print(blk_weight.shape, "\n")
blk_weight.tofile('./pre_weights/'+str(layer)+'_block'+'.bin')